In [1]:
pip install lxml bs4 html5lib

Note: you may need to restart the kernel to use updated packages.


In [2]:
conda update -n base -c defaults conda

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Majdi\anaconda3

  added / updated specs:
    - conda


The following packages will be REMOVED:

  python_abi-3.7-1_cp37m

The following packages will be SUPERSEDED by a higher-priority channel:

  conda              conda-forge::conda-4.9.2-py37h03978a9~ --> pkgs/main::conda-4.9.2-py37haa95532_0


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe



Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Majdi\anaconda3

  added / updated specs:
    - geopy


The following NEW packages will be INSTALLED:

  python_abi         conda-forge/win-64::python_abi-3.7-1_cp37m

The following packages will be SUPERSEDED by a higher-priority channel:

  conda               pkgs/main::conda-4.9.2-py37haa95532_0 --> conda-forge::conda-4.9.2-py37h03978a9_0


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Solving environment: ...working... done

# All requested packages already installed.



In [4]:
to_hoods = pd.read_html('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969')[0]

In [5]:
to_hoods = to_hoods[to_hoods['Borough']!='Not assigned']

In [6]:
to_hoods['Neighbourhood']=np.where(to_hoods['Neighbourhood']=='Not assigned',to_hoods['Borough'],to_hoods['Neighbourhood']) #using np.where(condition,y,n) to replace any 'Not assigned' values in the Neighbourhood column with the values in the Borough column

In [7]:
to_hoods = to_hoods.rename(columns={'Neighbourhood' : 'Neighborhood'})

In [8]:
to_hoods.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
to_hoods.shape

(103, 3)

In [10]:
to_hoods = to_hoods.groupby(['Postal Code'], sort=False).agg(', '.join)


In [11]:
to_hoods.reset_index(drop= False)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [12]:
to_hoods

,Borough,Neighborhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...
M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
M4Y,Downtown Toronto,Church and Wellesley
M7Y,East Toronto,"Business reply mail Processing Centre, South C..."


In [13]:
lat_lng_coords = pd.read_csv('https://cocl.us/Geospatial_data')
lat_lng_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
to_hoods_coords = pd.merge(to_hoods,lat_lng_coords,on='Postal Code')
to_hoods_coords.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [15]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [16]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[43.7, -79.3839300], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(to_hoods_coords['Latitude'], to_hoods_coords['Longitude'], to_hoods_coords['Borough'], to_hoods_coords['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [17]:
CLIENT_ID = 'PVM454TJSFO2OJDHJJCDXHNPFFVD30T4FZFBRBO5YKAQ1E4G' # your Foursquare ID
CLIENT_SECRET = '3EA21F25FU2XBVCUVJSDUCFI1IQEUKRWUZ2IPA240DGSNDOF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PVM454TJSFO2OJDHJJCDXHNPFFVD30T4FZFBRBO5YKAQ1E4G
CLIENT_SECRET:3EA21F25FU2XBVCUVJSDUCFI1IQEUKRWUZ2IPA240DGSNDOF


In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
toronto_venues = getNearbyVenues(names=to_hoods_coords['Neighborhood'],
                                   latitudes=to_hoods_coords['Latitude'],
                                   longitudes=to_hoods_coords['Longitude']
                                  );

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [20]:
print(toronto_venues.shape)
toronto_venues.head()

(4880, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café
2,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
3,Parkwoods,43.753259,-79.329656,A&W,43.760643,-79.326865,Fast Food Restaurant
4,Parkwoods,43.753259,-79.329656,Bruno's valu-mart,43.746143,-79.324630,Grocery Store


In [21]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,48,48,48,48,48,48
"Alderwood, Long Branch",24,24,24,24,24,24
"Bathurst Manor, Wilson Heights, Downsview North",31,31,31,31,31,31
Bayview Village,17,17,17,17,17,17
"Bedford Park, Lawrence Manor East",39,39,39,39,39,39
...,...,...,...,...,...,...
"Willowdale, Willowdale West",12,12,12,12,12,12
Woburn,9,9,9,9,9,9
Woodbine Heights,28,28,28,28,28,28


In [22]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 332 uniques categories.


In [23]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
toronto_onehot.shape

(4880, 332)

In [25]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025641,...,0.0,0.025641,0.0,0.0,0.0,0.0,0.0,0.025641,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
94,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
95,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.035714,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
96,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [26]:
toronto_grouped.shape

(98, 332)

In [27]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                  venue  freq
0    Chinese Restaurant  0.15
1         Shopping Mall  0.08
2  Caribbean Restaurant  0.04
3                Bakery  0.04
4           Pizza Place  0.04


----Alderwood, Long Branch----
            venue  freq
0  Discount Store  0.12
1            Park  0.08
2     Pizza Place  0.08
3             Pub  0.04
4           Trail  0.04


----Bathurst Manor, Wilson Heights, Downsview North----
               venue  freq
0  Convenience Store  0.06
1        Coffee Shop  0.06
2               Park  0.06
3               Bank  0.06
4     Ice Cream Shop  0.03


----Bayview Village----
                 venue  freq
0        Grocery Store  0.12
1         Intersection  0.12
2  Japanese Restaurant  0.12
3          Gas Station  0.12
4                 Bank  0.12


----Bedford Park, Lawrence Manor East----
                  venue  freq
0    Italian Restaurant  0.08
1           Coffee Shop  0.08
2  Fast Food Restaurant  0.05
3                  Bank  0.05
4        Sa

                venue  freq
0       Grocery Store  0.06
1  Italian Restaurant  0.06
2                Park  0.06
3         Coffee Shop  0.06
4     Thai Restaurant  0.05


----New Toronto, Mimico South, Humber Bay Shores----
           venue  freq
0           Park  0.12
1     Restaurant  0.06
2         Bakery  0.06
3   Skating Rink  0.06
4  Grocery Store  0.06


----North Park, Maple Leaf Park, Upwood Park----
                      venue  freq
0               Coffee Shop  0.18
1         Convenience Store  0.09
2        Dim Sum Restaurant  0.09
3  Mediterranean Restaurant  0.09
4               Gas Station  0.09


----North Toronto West, Lawrence Park----
                 venue  freq
0          Coffee Shop  0.09
1   Italian Restaurant  0.06
2   Mexican Restaurant  0.04
3      Thai Restaurant  0.04
4  Sporting Goods Shop  0.04


----Northwest, West Humber - Clairville----
                  venue  freq
0                 Hotel  0.33
1           Coffee Shop  0.33
2                Lounge  0.33


            venue  freq
0     Coffee Shop  0.11
1            Park  0.07
2     Pizza Place  0.07
3            Café  0.07
4  Sandwich Place  0.07


----York Mills West----
           venue  freq
0     Restaurant  0.20
1           Park  0.10
2    Coffee Shop  0.10
3  Grocery Store  0.05
4   Tennis Court  0.05


----York Mills, Silver Hills----
          venue  freq
0          Park  0.75
1          Pool  0.25
2           Zoo  0.00
3   Music Venue  0.00
4  Optical Shop  0.00




In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Shopping Mall,Pizza Place,Bakery,Caribbean Restaurant,Sandwich Place,Sri Lankan Restaurant,Restaurant,Sushi Restaurant,Supermarket
1,"Alderwood, Long Branch",Discount Store,Pizza Place,Park,Dance Studio,Moroccan Restaurant,Liquor Store,Shopping Mall,Donut Shop,Sandwich Place,Bagel Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Convenience Store,Bank,Park,Community Center,Diner,Dog Run,Frozen Yogurt Shop,Fried Chicken Joint,Sushi Restaurant
3,Bayview Village,Intersection,Bank,Japanese Restaurant,Gas Station,Grocery Store,Restaurant,Chinese Restaurant,Café,Shopping Mall,Skating Rink
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Sandwich Place,Fast Food Restaurant,Bank,Intersection,Sports Club,Indian Restaurant,Thai Restaurant,Park


In [30]:
neighborhoods_venues_sorted.shape

(98, 11)

In [31]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters,init='k-means++', random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 1, 1, 1, 1, 1, 1])

In [32]:

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = to_hoods_coords

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3.0,Park,Pharmacy,Bus Stop,Shopping Mall,Supermarket,Chinese Restaurant,Tennis Court,Caribbean Restaurant,Café,Laundry Service
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Coffee Shop,Portuguese Restaurant,Gym / Fitness Center,Playground,Pizza Place,Men's Store,French Restaurant,Lounge,Golf Course,Intersection
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Theater,Café,Restaurant,Park,Sushi Restaurant,Bakery,Breakfast Spot,Diner,Pub
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Furniture / Home Store,Restaurant,Fast Food Restaurant,Coffee Shop,Clothing Store,Dessert Shop,Sushi Restaurant,Fried Chicken Joint,Vietnamese Restaurant,Bowling Alley
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1.0,Coffee Shop,Sushi Restaurant,Park,Thai Restaurant,Japanese Restaurant,Italian Restaurant,Café,Ramen Restaurant,Pizza Place,Dance Studio


In [33]:
toronto_merged[toronto_merged['Cluster Labels'].isna()]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
95,M1X,Scarborough,Upper Rouge,43.836125,-79.205636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
toronto_merged.dropna(subset = ["Cluster Labels"], inplace=True)

In [35]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
#x = np.arange(kclusters)
#ys = [i + x + (i*x)**2 for i in range(kclusters)]
#colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#rainbow = [colors.rgb2hex(i) for i in colors_array]
rainbow = ['black', 'orange', 'red', 'blue', 'yellow']
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [36]:
map_clusters.save("map_clusters.html")

# Cluster 1

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,"Cliffside, Cliffcrest, Scarborough Village West",0.0,Pizza Place,Ice Cream Shop,Beach,Restaurant,Sports Bar,Auto Garage,Park,Event Space,Eastern European Restaurant,Electronics Store


# Cluster 2: The Suburbs

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Victoria Village,1.0,Coffee Shop,Portuguese Restaurant,Gym / Fitness Center,Playground,Pizza Place,Men's Store,French Restaurant,Lounge,Golf Course,Intersection
2,"Regent Park, Harbourfront",1.0,Coffee Shop,Theater,Café,Restaurant,Park,Sushi Restaurant,Bakery,Breakfast Spot,Diner,Pub
3,"Lawrence Manor, Lawrence Heights",1.0,Furniture / Home Store,Restaurant,Fast Food Restaurant,Coffee Shop,Clothing Store,Dessert Shop,Sushi Restaurant,Fried Chicken Joint,Vietnamese Restaurant,Bowling Alley
4,"Queen's Park, Ontario Provincial Government",1.0,Coffee Shop,Sushi Restaurant,Park,Thai Restaurant,Japanese Restaurant,Italian Restaurant,Café,Ramen Restaurant,Pizza Place,Dance Studio
6,"Malvern, Rouge",1.0,Fast Food Restaurant,Trail,Coffee Shop,Hobby Shop,Paper / Office Supplies Store,Supermarket,Bank,Bakery,Spa,Chinese Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...
98,"The Kingsway, Montgomery Road, Old Mill North",1.0,Coffee Shop,Sushi Restaurant,Pizza Place,Pub,Dessert Shop,Italian Restaurant,Breakfast Spot,French Restaurant,Bank,Burger Joint
99,Church and Wellesley,1.0,Coffee Shop,Italian Restaurant,Sushi Restaurant,Park,Hotel,Japanese Restaurant,Café,Dance Studio,Diner,Pizza Place
100,"Business reply mail Processing Centre, South C...",1.0,Park,Pizza Place,Brewery,Sushi Restaurant,Italian Restaurant,Coffee Shop,Fast Food Restaurant,Bakery,Breakfast Spot,Snack Place
101,"Old Mill South, King's Mill Park, Sunnylea, Hu...",1.0,Park,Bus Stop,Eastern European Restaurant,Shopping Mall,Ice Cream Shop,Italian Restaurant,Gym / Fitness Center,Yoga Studio,Electronics Store,Elementary School


# Cluster 3: City Centre + Yonge Street + Universities + Subway Line

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,"York Mills, Silver Hills",2.0,Park,Pool,Yoga Studio,Event Space,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Entertainment Service,Escape Room


# Cluster 4: The Airport

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,3.0,Park,Pharmacy,Bus Stop,Shopping Mall,Supermarket,Chinese Restaurant,Tennis Court,Caribbean Restaurant,Café,Laundry Service
5,"Islington Avenue, Humber Valley Village",3.0,Pharmacy,Playground,Skating Rink,Bakery,Bank,Café,Golf Course,Shopping Mall,Grocery Store,Park
8,"Parkview Hill, Woodbine Gardens",3.0,Pizza Place,Brewery,Intersection,Pharmacy,Pet Store,Coffee Shop,Rock Climbing Spot,Café,Fast Food Restaurant,Soccer Stadium
10,Glencairn,3.0,Grocery Store,Pizza Place,Italian Restaurant,Fast Food Restaurant,Park,Coffee Shop,Restaurant,Gas Station,Pet Store,Sandwich Place
11,"West Deane Park, Princess Gardens, Martin Grov...",3.0,Pizza Place,Convenience Store,Park,Bank,Restaurant,Gym,Clothing Store,Fish & Chips Shop,Hotel,Mexican Restaurant
16,Humewood-Cedarvale,3.0,Pizza Place,Convenience Store,Coffee Shop,Grocery Store,Soccer Stadium,Bagel Shop,Bank,Korean Restaurant,Middle Eastern Restaurant,Field
18,"Guildwood, Morningside, West Hill",3.0,Pizza Place,Bank,Fast Food Restaurant,Restaurant,Coffee Shop,Food & Drink Shop,Fried Chicken Joint,Discount Store,Supermarket,Liquor Store
21,Caledonia-Fairbanks,3.0,Pharmacy,Bus Stop,Pizza Place,Cosmetics Shop,Bakery,Beer Store,Falafel Restaurant,Discount Store,Japanese Restaurant,Grocery Store
26,Cedarbrae,3.0,Gas Station,Pharmacy,Coffee Shop,Bakery,Bank,Indian Restaurant,Yoga Studio,Burger Joint,Fried Chicken Joint,Sporting Goods Shop
27,Hillcrest Village,3.0,Pharmacy,Park,Coffee Shop,Convenience Store,Recreation Center,Shopping Mall,Korean Restaurant,Sandwich Place,Bank,Bakery


# Cluster 5: The Scarborough Bluffs

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,"Northwest, West Humber - Clairville",4.0,Hotel,Coffee Shop,Lounge,Yoga Studio,Falafel Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Entertainment Service,Escape Room
